<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/mercado_libre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías necesarias

In [431]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import get_column_letter, column_index_from_string

In [432]:
archivo = 'Resultados ML.xlsx'

# Archivos necesarios

In [433]:
dfm = pd.read_excel('MercadoLibre.xlsx', sheet_name=0, header=5)
renglonaje = pd.read_excel('Renglonaje.xlsx', sheet_name=0, header=0)
envios = pd.read_excel('Envios.xlsx', sheet_name=0, header=3)

## Extracción de precios del renglonaje

In [434]:
renglonaje.rename(columns={'Total que paga Edy a Truper contando reserva en garantía CON IVA': 'Costo a Truper'}, inplace=True)
precios_sku = renglonaje[['clave', 'Costo a Truper']]
precios_sku = precios_sku.loc[precios_sku['Costo a Truper'] != 'No disponible']
precios_sku['Costo a Truper'] = precios_sku['Costo a Truper'].astype(float)

In [435]:
precios_sku.rename(columns={'clave': 'SKU'}, inplace=True)

In [436]:
precios_codigo = renglonaje[['código', 'Costo a Truper']]
precios_codigo = precios_codigo.loc[precios_codigo['Costo a Truper'] != 'No disponible']
precios_codigo['Costo a Truper'] = precios_codigo['Costo a Truper'].astype(float)
precios_codigo.rename(columns={'código': 'SKU'}, inplace=True)
precios_codigo['SKU'] = precios_codigo['SKU'].astype(str)
precios_codigo.head()

,SKU,Costo a Truper
0,100048,290.92460
1,100049,407.29444
2,100098,13.96797
3,100099,24.82530
4,100101,65.94158


## Extracción de costos de envío

In [437]:
orden = envios['NUMERO ORDEN']
orden = orden.str.split('-')

In [438]:
orden = orden.str[1]

In [439]:
envios['NUMERO ORDEN'] = orden

In [440]:
envios['NUMERO ORDEN'] = envios['NUMERO ORDEN'].replace('', np.nan)
envios = envios.dropna(subset=['NUMERO ORDEN'])

In [441]:
publicacion = envios[['NUMERO ORDEN', 'VALOR GUIA ($)', 'ZONA EXTENDIDA ($)', 'EXTRAGRANDE ($)', 'SOBREPESOS ($)', 'TOTAL GUIA ($)']]

In [442]:
publicacion.rename(columns={'NUMERO ORDEN': '# de venta'}, inplace=True)

/tmp/ipython-input-442-1123234388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publicacion.rename(columns={'NUMERO ORDEN': '# de venta'}, inplace=True)


# Costo que Edgar paga a Truper

In [443]:
df = dfm.iloc[:,:15]
df2 = dfm.iloc[:,15:]
df2['# de venta'] = dfm['# de venta'].astype(str)

In [444]:
df[['Ingresos por envío (MXN) Pago del envio a cargo del comprador', 'Costos de envío (MXN)']] = df[['Ingresos por envío (MXN) Pago del envio a cargo del comprador', 'Costos de envío (MXN)']].fillna(0)

In [445]:
df = df.merge(precios_sku, on='SKU', how='left')
df = df.merge(precios_codigo, on='SKU', how='left')

In [446]:
df[['Costo a Truper_x', 'Costo a Truper_y']] = df[['Costo a Truper_x', 'Costo a Truper_y']].fillna(0)

In [447]:
df['Costo a Truper'] = df['Costo a Truper_x'] + df['Costo a Truper_y']
df = df.drop(['Costo a Truper_x', 'Costo a Truper_y'], axis=1)

In [448]:
df['Costo a Truper'] = df['Costo a Truper'] * df['Unidades']

# Costo de la guía Skydropx

In [449]:
df['# de venta'] = df['# de venta'].astype(str)

In [450]:
df = df.merge(publicacion, on='# de venta', how='left')

# Calculo de rentabilidad

In [451]:
df.iloc[:,15:] = df.iloc[:,15:].fillna(0)

/tmp/ipython-input-451-1814211588.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.iloc[:,15:] = df.iloc[:,15:].fillna(0)


In [452]:
df['Rentabilidad'] = df['Total (MXN)'] - df['Costo a Truper'] - df['TOTAL GUIA ($)']

In [453]:
df.loc[df['Costo a Truper'] == 0, 'Rentabilidad'] = 0

In [454]:
df.loc[df['Anulaciones y reembolsos (MXN)'] < 0, 'Rentabilidad'] = 0

In [455]:
df['¿Rentable?'] = ''
df.loc[df['Rentabilidad'] > 0, '¿Rentable?'] = 'Sí'
df.loc[df['Rentabilidad'] <= 0, '¿Rentable?'] = 'No'

In [456]:
df.shape

(926, 23)

In [457]:
df = df.merge(df2, on='# de venta', how='left')

# Limpieza de datos

In [458]:
df = df.drop_duplicates(subset=['# de venta'])

# Creacion del excel

In [459]:
df.to_excel(archivo, sheet_name='Rentabilidad', index=False)

# Formato con Openpyxl

In [460]:
wb = load_workbook(archivo)
ws = wb['Rentabilidad']

## Columnas por índice

In [461]:
total = column_index_from_string('M')
costo = column_index_from_string('P')
envio = column_index_from_string('U')
rentabilidad = column_index_from_string('V')
rentable = column_index_from_string('W')
c_final = column_index_from_string('BO')

## Colores de fondo

En orden, todos con tonalidades claras:
- naranja
- verde
- rojo
- azul
- morado
- gris
- amarillo
- negro

In [462]:
fill_o = PatternFill(start_color='f9cb9c', end_color='f9cb9c', fill_type='solid')
fill_lg = PatternFill(start_color='b6d7a8', end_color='b6d7a8', fill_type='solid')
fill_lr = PatternFill(start_color='ea9999', end_color='ea9999', fill_type='solid')
fill_lb = PatternFill(start_color='a4c2f4', end_color='a4c2f4', fill_type='solid')
fill_p = PatternFill(start_color='b4a7d6', end_color='b4a7d6', fill_type='solid')
fill_gy = PatternFill(start_color='cccccc', end_color='cccccc', fill_type='solid')
fill_y = PatternFill(start_color='ffe599', end_color='ffe599', fill_type='solid')
fill_b = PatternFill(start_color='000000', end_color='000000', fill_type='solid')

## Fuentes

In [463]:
font_w = Font(name='Calibri', size=12, color = 'ffffff', bold=True)
font_b = Font(name='Calibri', size=12, color = '000000', bold=True)
font_t = Font(name='Calibri', size=12, color = '000000', bold=False)

## Dando formato

### Color a columnas

In [464]:
for row in range(1, 1000):
  if ws.cell(row=row, column=total).value == None:
    break
  ws.cell(row=row, column=total).fill = fill_y
  ws.cell(row=row, column=costo).fill = fill_o
  ws.cell(row=row, column=envio).fill = fill_lb
  for col in range(costo + 1, envio):
    ws.cell(row=row, column=col).fill = fill_p

### Color rentabilidad

In [465]:
for row in range(2, 1000):
  if ws.cell(row=row, column=rentabilidad).value == None:
    break
  if ws.cell(row=row, column=rentabilidad).value > 0:
    ws.cell(row=row, column=rentabilidad).fill = fill_lg
    ws.cell(row=row, column=rentable).fill = fill_lg
  else:
    ws.cell(row=row, column=rentabilidad).fill = fill_lr
    ws.cell(row=row, column=rentable).fill = fill_lr

### Fuente a columnas

### Formato numerico

In [466]:
num_format = '$ #,##0.00'

In [467]:
for row in range(2, 1000):
  for col in range(8, 12):
    ws.cell(row=row, column=col).number_format = num_format
  ws.cell(row=row, column=total).number_format = num_format
  for col in range(costo, rentabilidad+1):
    ws.cell(row=row, column=col).number_format = num_format

### Claridad visual del documento

In [468]:
for row in range(2, 1000):
  for col in range(8, 22):
    if ws.cell(row=row, column=col).value == 0:
      ws.cell(row=row, column=col).value = None

In [469]:
for col in range(1, c_final):
  ws.column_dimensions[get_column_letter(col)].width = 14
  ws.cell(row=1, column=col).font = font_b

In [470]:
ws.cell(row=1, column=rentabilidad).fill = fill_b
ws.cell(row=1, column=rentabilidad).font = font_w
ws.cell(row=1, column=rentable).fill = fill_b
ws.cell(row=1, column=rentable).font = font_w

In [471]:
for row in range(2, 1000):
  for col in range(1, c_final):
    ws.cell(row=row, column=col).font = font_t

# Guardando archivo

In [472]:
wb.save(archivo)